In [51]:
# import datetime
# import os.path
# import pickle
# from google.auth.transport.requests import Request
# from google_auth_oauthlib.flow import InstalledAppFlow
# from googleapiclient.discovery import build

# # Scope
# SCOPES = ['https://www.googleapis.com/auth/calendar']

# creds = None

# # The file token.pickle stores the user's access and refresh tokens
# # It is created automatically when the authorization flow completes for the first time
# if os.path.exists('token.pickle'):
#     with open('token.pickle', 'rb') as token:
#         creds = pickle.load(token)

# # If no valid credentials are available, let the user log in
# if not creds or not creds.valid:
#     if creds and creds.expired and creds.refresh_token:
#         creds.refresh(Request())
#     else:
#         flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
#         creds = flow.run_local_server(port=0)
#     with open('token.pickle', 'wb') as token:
#         pickle.dump(creds, token)

# service = build('calendar', 'v3', credentials=creds)

# # TODO: Convert time ISO to EET
# # Call the Calendar API
# now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
# print('Getting the upcoming 10 events')
# events_result = service.events().list(calendarId='primary', timeMin=now,
#                                       maxResults=10, singleEvents=True,
#                                       orderBy='startTime').execute()
# events = events_result.get('items', [])

# if not events:
#     print('No upcoming events found.')
# else:
#     for event in events:
#         start = event['start'].get('dateTime', event['start'].get('date'))
#         print(start, event['summary'])


Getting the upcoming 10 events
2024-03-27 Discussion with Daniela
2024-03-28 DL P2 Alg. de Cautare
2024-03-28T09:00:00+02:00 Gym Upper
2024-03-28T12:00:00+02:00 emag
2024-03-28T13:00:00+02:00 Vine Mancarea
2024-03-28T13:45:00+02:00 Brand Minds Buy
2024-03-28T14:00:00+02:00 Pantaloni Croitor (cei negri)
2024-03-28T18:00:00+02:00 Lab PSS
2024-03-29T09:30:00+02:00 RL for SW Testing & Game Dev
2024-03-31T09:00:00+03:00 Tenis


In [1]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import os
import datetime

SCOPES = ['https://www.googleapis.com/auth/calendar']

def get_google_calendar_service():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return build('calendar', 'v3', credentials=creds)

def get_events_for_date(date, service):
    time_min = date.isoformat() + 'T00:00:00Z'
    time_max = date.isoformat() + 'T23:59:59Z'
    events_result = service.events().list(
        calendarId='primary', timeMin=time_min, timeMax=time_max,
        singleEvents=True, orderBy='startTime'
    ).execute()
    return events_result.get('items', [])

service = get_google_calendar_service()
date = datetime.date.today()  # or any other date
events = get_events_for_date(date, service)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=155962034767-njrtlrhi56mj5a2j7csotr6gvjkqk26m.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A50258%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=aVvlCpVwu5aGUqooIk6yo7vRhT0V1L&access_type=offline


In [2]:
def format_events(events):
    if not events:
        return "No events for the day."
    formatted_events = ["Event: {} at {}".format(event['summary'], event['start'].get('dateTime', event['start'].get('date'))) for event in events]
    return "\n".join(formatted_events)

formatted_context = format_events(events)


In [3]:
formatted_context

'Event: Discussion with Daniela at 2024-03-27\nEvent: Eat at 2024-03-27T10:00:00+02:00\nEvent: eMAG at 2024-03-27T11:00:00+02:00\nEvent: emag at 2024-03-27T12:00:00+02:00\nEvent: Eat at 2024-03-27T13:30:00+02:00\nEvent: eMAG at 2024-03-27T15:00:00+02:00\nEvent: DL P2 Alg. de Cautare at 2024-03-28'

In [4]:
# Setup Client
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(
    organization=os.environ.get("organization"),
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [5]:
def query_openai(prompt, formatted_context):
    # Chunked text, streamed, closer to IRL
    stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful scheduling assistant. You should always be polite and concise."},
            {"role": "assistant", "content": f"{formatted_context}"},
            {"role": "user", "content": f"{prompt}"},
        ],
        stream=True,
    )
    for chunk in stream:
        print(chunk.choices[0].delta.content or "", end="")
    return stream

prompt = "Given the following events for today:\n{}\nCan I schedule a meeting for 1 hour today? If yes, what time?"
response = query_openai(prompt, formatted_context)
# print(response)

Based on the events for today, you have the following available time slots for a 1-hour meeting:
- From 10:00 AM to 11:00 AM
- From 1:30 PM to 2:30 PM

In [6]:
print(response)

In [7]:
prompt

'Given the following events for today:\n{}\nCan I schedule a meeting for 1 hour today? If yes, what time?'

In [8]:
from datetime import datetime, timedelta
import pytz

SCOPES = ['https://www.googleapis.com/auth/calendar']

def create_event(service, event_name, start_datetime, end_datetime, timezone='UTC'):
    """
    Create an event in the Google Calendar.

    Args:
        service: Authorized Google Calendar service instance.
        event_name: Name of the event.
        start_datetime: Start datetime of the event in ISO format.
        end_datetime: End datetime of the event in ISO format.
        timezone: Timezone of the event.

    Returns:
        The created event.
    """

    event = {
        'summary': event_name,
        'start': {
            'dateTime': start_datetime.isoformat(),
            'timeZone': timezone,
        },
        'end': {
            'dateTime': end_datetime.isoformat(),
            'timeZone': timezone,
        },
    }

    created_event = service.events().insert(calendarId='primary', body=event).execute()
    return created_event



# Example usage:
service = get_google_calendar_service()
event_name = "Meeting with Bob"
start_datetime = datetime(2024, 3, 28, 15, 0, 0)  # Example start datetime
end_datetime = start_datetime + timedelta(hours=1)  # Example end datetime (1 hour after start)

# Create the event
created_event = create_event(service, event_name, start_datetime, end_datetime)

print("Event created:", created_event.get('htmlLink'))


Event created: https://www.google.com/calendar/event?eid=bDJ2amhtZGJhbW1hNjU3bnBkMGJsYzd2OTggc2VyZ2l1Y3JhaW92ZWFudUBt


### Tools

In [47]:
!pip install langchain-openai

  Using cached langchain_core-0.1.35-py3-none-any.whl (273 kB)
  Using cached langsmith-0.1.33-py3-none-any.whl (86 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.87
    Uninstalling langsmith-0.0.87:
      Successfully uninstalled langsmith-0.0.87
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.23
    Uninstalling langchain-core-0.1.23:
      Successfully uninstalled langchain-core-0.1.23
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.0 requires langsmith<0.1.0,>=0.0.77, but you have langsmith 0.1.33 which is incompatible.
langchain-community 0.0.20 requires langsmith<0.1,>=0.0.83, but you have langsmith 0.1.33 which is incompatible.


In [53]:
!pip install -qU langchain openai transformers

In [54]:
from langchain_core.pydantic_v1 import BaseModel, Field

In [55]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

TypeError: issubclass() arg 1 must be a class

In [45]:

print(dir(langchain))

['Any', 'Optional', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__getattr__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_warn_on_import', 'pydantic_v1', 'surface_langchain_deprecation_warnings', 'warnings']


In [57]:
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type, Optional, List
from googleapiclient.discovery import build

class AvailabilityInput(BaseModel):
    date: str = Field(description="Date for checking availability in YYYY-MM-DD format")

class AvailabilityCheckerTool(BaseTool):
    name = "availability_checker"
    description = "Checks available times in Google Calendar for a given date"
    args_schema: Type[BaseModel] = AvailabilityInput

    def _run(self, date: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        """Check availability on a specific date."""
        service = get_google_calendar_service()
        date_obj = datetime.strptime(date, '%Y-%m-%d')
        events = get_events_for_date(date_obj, service)

        # Assuming the work hours are 9 AM to 5 PM
        work_hours = set(range(9, 17))
        for event in events:
            start = event['start'].get('dateTime', event['start'].get('date'))
            end = event['end'].get('dateTime', event['end'].get('date'))
            start_hour = datetime.fromisoformat(start).hour
            end_hour = datetime.fromisoformat(end).hour
            for hour in range(start_hour, end_hour):
                work_hours.discard(hour)

        available_hours = sorted(list(work_hours))
        availability = ", ".join(f"{hour}:00" for hour in available_hours)
        return "Available hours on {}: {}".format(date, availability if availability else "No available hours")


TypeError: issubclass() arg 1 must be a class